In [4]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.functions import when, col, sum, count, isnan, round
from pyspark.sql.functions import regexp_replace, concat_ws, sha2, rtrim, substring
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import year, month, dayofmonth, quarter
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import trim, regexp_replace, when, col

import os
import re

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [5]:
def salvar_df(df, file):
    output = "input/projeto-hive/gold/" + file
    erase = "hdfs dfs -rm " + output + "/*"
    rename = "hdfs dfs -get /datalake/gold/"+file+"/part-* input/projeto-hive/gold/"+file+".csv"
    
    print(rename)    
    
    df.coalesce(1).write\
        .format("csv")\
        .option("header", True)\
        .option("delimiter", ";")\
        .mode("overwrite")\
        .save("/datalake/gold/"+file+"/")
    
    os.system(erase)
    os.system(rename)

In [6]:
# Carregar tabela endereco e remover a primeira linha que copiou o cabeçalho
df_endereco = spark.read.table("desafio_curso.endereco")
rdd = df_endereco.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_endereco = rdd.toDF(df_endereco.schema)

# Carregar tabela clientes e remover a primeira linha que copiou o cabeçalho
df_clientes = spark.read.table("desafio_curso.clientes")
rdd = df_clientes.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_clientes = rdd.toDF(df_clientes.schema)

# Carregar tabela divisao e remover a primeira linha que copiou o cabeçalho
df_divisao = spark.read.table("desafio_curso.divisao")
rdd = df_divisao.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_divisao = rdd.toDF(df_divisao.schema)

# Carregar tabela regiao e remover a primeira linha que copiou o cabeçalho
df_regiao = spark.read.table("desafio_curso.regiao")
rdd = df_regiao.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_regiao = rdd.toDF(df_regiao.schema)

# Carregar tabela regiao e remover a primeira linha que copiou o cabeçalho
df_vendas = spark.read.table("desafio_curso.vendas")
rdd = df_vendas.rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])
df_vendas = rdd.toDF(df_vendas.schema)


# df_vendas.limit(3).show()
# df_endereco.limit(3).show()
# df_clientes.limit(3).show()
# df_divisao.show()
# df_regiao.show()

In [7]:
# #Teste quantidade de linhas de cada tabela

# print(df_vendas.count())
# print(df_endereco.count())
# print(df_clientes.count())
# print(df_divisao.count())
# print(df_regiao.count())

In [8]:
# #APAGAR 
# from pyspark.sql.functions import countDistinct

# distinct_count = df_endereco.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [9]:
#limpar linhas duplicadas em clientes e endereço
df_clientes = df_clientes.dropDuplicates(['customer_key'])
df_endereco = df_endereco.dropDuplicates(['address_number'])

In [10]:
# #APAGAR 

# distinct_count = df_endereco.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [11]:
#Alterar nome colunas
df_clientes = df_clientes.withColumnRenamed("customer_key", "customer_key_cliente")
df_endereco = df_endereco.withColumnRenamed("address_number", "address_number_endereco")
df_divisao = df_divisao.withColumnRenamed("division", "division_divisao")
df_regiao = df_regiao.withColumnRenamed("region_code", "region_code_regiao")


#Fazer um tabelão

df_stage = df_vendas.join(df_clientes,df_vendas.customer_key == df_clientes.customer_key_cliente,"left")
df_stage = df_stage.join(df_endereco,df_stage.address_number == df_endereco.address_number_endereco,"left")
df_stage = df_stage.join(df_divisao,df_stage.division == df_divisao.division_divisao,"left")
df_stage = df_stage.join(df_regiao,df_stage.region_code == df_regiao.region_code_regiao,"left")


# df_stage.limit(3).show()

In [12]:
# #Alterar nome colunas
# df_clientes = df_clientes.withColumnRenamed("customer_key", "customer_key_cliente")
# # df_endereco = df_endereco.withColumnRenamed("address_number", "address_number_endereco")
# df_divisao = df_divisao.withColumnRenamed("division", "division_divisao")
# df_regiao = df_regiao.withColumnRenamed("region_code", "region_code_regiao")


# #Fazer um tabelão

# # df_stage = df_vendas.join(df_clientes,df_vendas.customer_key == df_clientes.customer_key_cliente,"left")

# df_vendas_enderecos = df_vendas.join(df_endereco,df_vendas.customer_key == df_endereco.address_number,"left")



# df_clientes_divisao = df_clientes.join(df_divisao,df_clientes.division == df_divisao.division_divisao,"left")
# df_clientes_divisao_regiao = df_clientes_divisao.join(df_regiao,df_clientes_divisao.region_code == df_regiao.region_code_regiao,"left")

# df_stage = df_vendas_enderecos.join(df_clientes_divisao_regiao,df_vendas_enderecos.customer_key == df_clientes_divisao_regiao.address_number,"left")

# df_stage.limit(3).show()

In [13]:
# # Somar os valores da coluna "sales_amount" no DataFrame "df_stage"
# total_sales_amount = df_stage.agg(sum("sales_amount")).collect()[0][0]
# total_sales_quantity = df_stage.agg(sum("sales_quantity")).collect()[0][0]

# # Imprimir o resultado
# print("Total sales amount:", total_sales_amount)
# print("Total sales quantity:", total_sales_quantity)

# print(df_vendas.count())
# print(df_clientes.count())
# print(df_stage.count())

In [14]:
# #APAGAR 

# distinct_enderecos = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_enderecos)
# distinct_enderecos = df_stage.select(countDistinct("customer_key")).collect()[0][0]
# print(distinct_enderecos)
# distinct_divisoes = df_stage.select(countDistinct("division")).collect()[0][0]
# print(distinct_divisoes)
# distinct_regioes = df_stage.select(countDistinct("region_code")).collect()[0][0]
# print(distinct_regioes)
# distinct_vendas = df_stage.select(countDistinct("order_number")).collect()[0][0]
# print(distinct_vendas)

In [15]:
#Adicionar colunas Ano, Mês, Dia, Trimestre tomando como base a coluna invoice_date

df_stage = df_stage.withColumn('invoice_date', to_date(col('invoice_date'), 'dd/MM/yyyy'))

df_stage = df_stage \
    .withColumn('Ano', year('invoice_date')) \
    .withColumn('Mes', month('invoice_date')) \
    .withColumn('Dia', dayofmonth('invoice_date')) \
    .withColumn('Trimestre', quarter('invoice_date'))

df_stage.show()    

+--------------------+------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+--------------------+-------------+----------------+--------------------+-------------+--------+----------------+------------+-----------+------------------+-----------+-----------------------+-------+-------+------------------+--------------------+------------------+--------------------+-----+--------+----------------+-------------+------------------+-----------+----+---+---+---------+
|actual_delivery_date|customer_key|  date_key|discount_amount|invoice_date|invoice_number|item_class|item_number|                item|line_number|list_price|order_number|promised_delivery_date|sales_amount|sales_amount_based_on_list_price|sales_cost_amount|sales_margin_amount|

In [16]:
# # #APAGAR 

# distinct_count = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [17]:
#Campos decimais ou inteiros nulos ou vazios, deversão ser preenchidos por 0.

cols_to_check = ['item_number', 'discount_amount', 'list_price', 'sales_amount', 'sales_amount_based_on_list_price', 'sales_cost_amount', 'sales_margin_amount', 'sales_price', 'line_number', 'sales_quantity']

for col_name in cols_to_check:
     df_stage = df_stage.withColumn(col_name, when(col(col_name) == '', 0).otherwise(col(col_name)))


In [18]:
# #APAGAR 

# distinct_count = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [19]:
#Filtro para verificar se determinado cliente que tinha valores vazio foi substituidos por zero
df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show(truncate=False)

+--------------------+------------+----------+---------------+------------+--------------+----------+-----------+------------------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+--------------------+-------------+----------------+--------------------+-------------+--------+----------------+------------+-----------+------------------+-----------+-----------------------+-------+-------+---------------------+----------------------------------------+----------------------------------------+----------------------------------------+-----+----------+----------------+-------------+------------------+-----------+----+---+---+---------+
|actual_delivery_date|customer_key|date_key  |discount_amount|invoice_date|invoice_number|item_class|item_number|item                          |line_number|list_price|order_number|promised_delivery_da

In [20]:
# #APAGAR 

# distinct_count = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [21]:
#Campos strings vazios deverão ser preenchidos com 'Não informado'
all_columns = df_stage.columns

for column in all_columns:
    df_stage = df_stage.withColumn(column, 
                                   when(trim(regexp_replace(col(column), '\n', '')) == "", "Nao informado")
                                   .otherwise(col(column)))

#Filtro para verificar se determinado cliente que tinha strings vaziasforam substituídos com 'Não informado'
df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show(truncate=False)


+--------------------+------------+----------+---------------+------------+--------------+-------------+-----------+------------------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+--------------------+-------------+----------------+--------------------+-------------+--------+----------------+------------+-----------+------------------+-----------+-----------------------+-------+-------+---------------------+------------------+------------------+------------------+-----+----------+----------------+-------------+------------------+-----------+----+---+---+---------+
|actual_delivery_date|customer_key|date_key  |discount_amount|invoice_date|invoice_number|item_class   |item_number|item                          |line_number|list_price|order_number|promised_delivery_date|sales_amount|sales_amount_based_on_list_price|sales_cost_

In [22]:
# #APAGAR 

# distinct_count = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [23]:
# #Verificar tamanho da tabela
# print(df_stage.count())

In [24]:
# df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show()


+--------------------+------------+----------+---------------+------------+--------------+-------------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+--------------------+-------------+----------------+--------------------+-------------+--------+----------------+------------+-----------+------------------+-----------+-----------------------+-------+-------+--------------------+------------------+------------------+------------------+-----+----------+----------------+-------------+------------------+-----------+----+---+---+---------+
|actual_delivery_date|customer_key|  date_key|discount_amount|invoice_date|invoice_number|   item_class|item_number|                item|line_number|list_price|order_number|promised_delivery_date|sales_amount|sales_amount_based_on_list_price|sales_cost_amount|sales_margin_a

In [25]:
# # Conta o número de valores vazios em todas as colunas
# empty_counts = [df_stage.select(when(col(c) == "", 1).otherwise(0)).agg({"*": "sum"}).collect()[0][0] for c in df_stage.columns]
# for i, c in enumerate(df_stage.columns):
#     print(f"{c}: {empty_counts[i]}")

In [26]:
# # Verificar se há valores nulos em colunas numéricas
# from pyspark.sql.functions import isnan, col
# from pyspark.sql.types import DoubleType, IntegerType, LongType, FloatType

# numeric_cols = [c.name for c in df_stage.schema if c.dataType in [DoubleType(), IntegerType(), LongType(), FloatType()]]

# null_counts = [df_stage.select(isnan(col(c)).alias(c)).agg({"*": "sum"}).collect()[0][0] for c in numeric_cols]
# for i, c in enumerate(numeric_cols):
#     print(f"{c}: {null_counts[i]}")

In [27]:
#Adicionar chaves estrangeiras
df_stage = df_stage.withColumn('PK_TEMPO', sha2(concat_ws("",df_stage.invoice_date, df_stage.Ano,df_stage.Mes,df_stage.Dia,df_stage.Trimestre), 256))

df_stage = df_stage.withColumn('PK_CLIENTES', sha2(concat_ws("",df_stage.customer_key,df_stage.customer,df_stage.business_family_name,df_stage.business_unit,df_stage.customer_type,df_stage.division,df_stage.line_of_business,df_stage.phone,df_stage.region_code,df_stage.regional_sales_mgr,df_stage.search_type), 256))

df_stage = df_stage.withColumn('PK_LOCALIDADE', sha2(concat_ws("",df_stage.address_number,df_stage.city,df_stage.country,df_stage.state,df_stage.zip_code,df_stage.division,df_stage.division_name,df_stage.region_code,df_stage.region_name,df_stage.customer_address_1,df_stage.customer_address_2,df_stage.customer_address_3,df_stage.customer_address_4), 256))

In [28]:
#APAGAR 

# distinct_count = df_stage.select(countDistinct("address_number")).collect()[0][0]
# print(distinct_count)

In [29]:
df_stage.createOrReplaceTempView("stage")

In [30]:
# df_stage.filter((col("customer_key") == "10016588") & (col("invoice_number") == "110753")).show()

In [31]:
# df_stage.show()

In [32]:
#Gerar dimensões e fato a partir do tabelão  
df_tempo = spark.sql("SELECT DISTINCT PK_TEMPO, invoice_date, Ano, Mes, Dia, Trimestre FROM stage")

In [33]:
#Exportar dataframes como tabelas csv
salvar_df(df_tempo, 'dim_tempo')

hdfs dfs -get /datalake/gold/dim_tempo/part-* input/projeto-hive/gold/dim_tempo.csv


In [34]:
df_clientes = spark.sql("SELECT DISTINCT PK_CLIENTES, customer_key, customer, business_family_name, business_unit, customer_type, division, line_of_business, phone, region_code, regional_sales_mgr, search_type FROM stage")

In [35]:
salvar_df(df_clientes, 'dim_clientes')

hdfs dfs -get /datalake/gold/dim_clientes/part-* input/projeto-hive/gold/dim_clientes.csv


In [36]:
df_localidade = spark.sql("SELECT DISTINCT PK_LOCALIDADE, address_number, city, country, state, zip_code, division, division_name, region_code, region_name, customer_address_1, customer_address_2, customer_address_3, customer_address_4 FROM stage")
# df_localidade.show()

+--------------------+--------------+------------+-------+-------------+-------------+--------+-------------+-----------+-------------+--------------------+--------------------+------------------+------------------+
|       PK_LOCALIDADE|address_number|        city|country|        state|     zip_code|division|division_name|region_code|  region_name|  customer_address_1|  customer_address_2|customer_address_3|customer_address_4|
+--------------------+--------------+------------+-------+-------------+-------------+--------+-------------+-----------+-------------+--------------------+--------------------+------------------+------------------+
|4f9d6b1a369aac5ac...|      10021270|        null|   null|         null|         null|       1|International|          5|International|                null|                null|              null|              null|
|2ec8bd45a27fad39f...|      10007457|        null|   null|         null|         null|       1|International|          5|International| 

In [37]:
salvar_df(df_localidade, 'dim_localidade')

hdfs dfs -get /datalake/gold/dim_localidade/part-* input/projeto-hive/gold/dim_localidade.csv


In [38]:
ft_vendas = spark.sql("SELECT PK_CLIENTES, PK_TEMPO, PK_LOCALIDADE, actual_delivery_date, date_key, discount_amount, invoice_date, invoice_number, item_class, item_number, item, line_number, list_price, order_number, promised_delivery_date, sales_amount, sales_amount_based_on_list_price, sales_cost_amount, sales_margin_amount, sales_price, sales_quantity, sales_rep, u_m  FROM stage")  

In [39]:
salvar_df(ft_vendas, 'ft_vendas')

hdfs dfs -get /datalake/gold/ft_vendas/part-* input/projeto-hive/gold/ft_vendas.csv


In [40]:
print(ft_vendas.count())
print(df_localidade.count())
print(df_clientes.count())
print(df_tempo.count())

65535
616
616
560


In [41]:
# Somar os valores da coluna "sales_amount" no DataFrame "df_stage"
total_sales_amount = ft_vendas.agg(sum("sales_amount")).collect()[0][0]
total_sales_quantity = ft_vendas.agg(sum("sales_quantity")).collect()[0][0]

# Imprimir o resultado
print("Total sales amount:", total_sales_amount)
print("Total sales quantity:", total_sales_quantity)

Total sales amount: 26409762.0
Total sales quantity: 2943194.0
